# 📈 Stock Market API Endpoints with Flask & Yahoo Finance

In this project, we will develop four robust API endpoints to facilitate seamless data retrieval and analysis:

1. **Company Information Endpoint** - Retrieve detailed company information
2. **Stock Market Data Endpoint** - Fetch real-time stock market data
3. **Historical Market Data Endpoint** - Return historical market data within a date range
4. **Analytical Insights Endpoint** - Perform comprehensive analysis and deliver actionable insights

## Installation

Before running this notebook, install the required dependencies:

In [1]:
!pip install flask yfinance pandas numpy

  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached peewee-3.18.3-cp313-cp313-macosx_15_0_arm64.whl
  Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.8 kB)
Using cached yfinance-0.2.66-py2.py3-none-any.whl (123 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.0 MB/s  0:00:00
Using cached websockets-15.0.1-cp313-cp313-macosx_11_0_arm64.whl (173 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [yfinance]4/6 [curl_cffi]




## Understanding Yahoo Finance (yfinance) Library

The `yfinance` library is a popular Python package that provides a simple interface to download market data from Yahoo Finance.

### Key Features:
- **Company Information**: Business summary, sector, industry, officers
- **Real-time Data**: Current price, market cap, volume
- **Historical Data**: OHLCV (Open, High, Low, Close, Volume) data
- **Financial Statements**: Income statements, balance sheets, cash flow

### Basic Usage:
```python
import yfinance as yf

# Create a Ticker object
ticker = yf.Ticker("AAPL")

# Get company info
info = ticker.info

# Get historical data
history = ticker.history(start="2024-01-01", end="2024-12-31")
```


## 1. Company Information Endpoint

This endpoint retrieves detailed company information such as:
- Full company name
- Business summary
- Industry and sector
- Key officers' names and titles

### Endpoint Details:
| Property | Value |
|----------|-------|
| **Route** | `/api/company/<symbol>` |
| **Method** | GET |
| **Input** | Company symbol (e.g., AAPL, GOOGL, MSFT) |
| **Output** | JSON with company details |


In [2]:
from flask import Flask, request, jsonify
import yfinance as yf

app = Flask(__name__)

@app.route('/api/company/<symbol>', methods=['GET'])
def get_company_info(symbol):
    """
    Retrieve detailed company information for a given stock symbol.
    
    Args:
        symbol (str): Stock ticker symbol (e.g., AAPL, GOOGL)
        
    Returns:
        JSON response with company information or error message
    """
    try:
        # Create a Ticker object for the given symbol
        ticker = yf.Ticker(symbol.upper())
        
        # Fetch company information
        info = ticker.info
        
        # Check if valid data was returned
        if not info or 'shortName' not in info:
            return jsonify({
                'success': False,
                'error': f'No data found for symbol: {symbol}'
            }), 404
        
        # Extract key officers information
        officers = []
        if 'companyOfficers' in info:
            for officer in info['companyOfficers'][:5]:  # Limit to top 5 officers
                officers.append({
                    'name': officer.get('name', 'N/A'),
                    'title': officer.get('title', 'N/A'),
                    'age': officer.get('age', 'N/A')
                })
        
        # Build response with company information
        company_data = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('longName', info.get('shortName', 'N/A')),
            'business_summary': info.get('longBusinessSummary', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'website': info.get('website', 'N/A'),
            'headquarters': {
                'city': info.get('city', 'N/A'),
                'state': info.get('state', 'N/A'),
                'country': info.get('country', 'N/A')
            },
            'employees': info.get('fullTimeEmployees', 'N/A'),
            'key_officers': officers
        }
        
        return jsonify(company_data), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve company information: {str(e)}'
        }), 500


if __name__ == '__main__':
    app.run(debug=False, port=5000)

# Testing with curl:
# curl http://localhost:5000/api/company/AAPL
# curl http://localhost:5000/api/company/GOOGL
# curl http://localhost:5000/api/company/MSFT


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Dec/2025 02:59:31] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Dec/2025 02:59:41] "GET /api/company/AAPL HTTP/1.1" 200 -


### Sample Response:

The endpoint returns a JSON response with detailed company information including the business summary, industry, sector, headquarters location, number of employees, and key officers.

---

## 2. Stock Market Data Endpoint

This endpoint fetches real-time stock market data including:
- Market state (open/closed)
- Current market price
- Price change and percentage change
- Volume, market cap, and other metrics

### Endpoint Details:
| Property | Value |
|----------|-------|
| **Route** | `/api/stock/<symbol>` |
| **Method** | GET |
| **Input** | Company symbol |
| **Output** | JSON with real-time stock data |


In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf

app = Flask(__name__)

@app.route('/api/stock/<symbol>', methods=['GET'])
def get_stock_data(symbol):
    """
    Fetch real-time stock market data for a given symbol.
    
    Args:
        symbol (str): Stock ticker symbol
        
    Returns:
        JSON response with current stock market data
    """
    try:
        # Create a Ticker object
        ticker = yf.Ticker(symbol.upper())
        info = ticker.info
        
        # Check if valid data was returned
        if not info or 'regularMarketPrice' not in info:
            return jsonify({
                'success': False,
                'error': f'No stock data found for symbol: {symbol}'
            }), 404
        
        # Calculate price change and percentage
        current_price = info.get('regularMarketPrice', 0)
        previous_close = info.get('regularMarketPreviousClose', 0)
        price_change = round(current_price - previous_close, 2) if current_price and previous_close else 0
        percentage_change = round((price_change / previous_close) * 100, 2) if previous_close else 0
        
        # Build stock data response
        stock_data = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('shortName', 'N/A'),
            'market_state': info.get('marketState', 'N/A'),
            'currency': info.get('currency', 'USD'),
            'price_data': {
                'current_price': current_price,
                'previous_close': previous_close,
                'price_change': price_change,
                'percentage_change': percentage_change,
                'day_high': info.get('dayHigh', 'N/A'),
                'day_low': info.get('dayLow', 'N/A'),
                'open_price': info.get('regularMarketOpen', 'N/A')
            },
            'volume_data': {
                'volume': info.get('regularMarketVolume', 'N/A'),
                'average_volume': info.get('averageVolume', 'N/A'),
                'average_volume_10day': info.get('averageVolume10days', 'N/A')
            },
            'valuation': {
                'market_cap': info.get('marketCap', 'N/A'),
                'enterprise_value': info.get('enterpriseValue', 'N/A'),
                'pe_ratio': info.get('trailingPE', 'N/A'),
                'forward_pe': info.get('forwardPE', 'N/A'),
                'price_to_book': info.get('priceToBook', 'N/A')
            },
            'range': {
                'fifty_two_week_high': info.get('fiftyTwoWeekHigh', 'N/A'),
                'fifty_two_week_low': info.get('fiftyTwoWeekLow', 'N/A'),
                'fifty_day_average': info.get('fiftyDayAverage', 'N/A'),
                'two_hundred_day_average': info.get('twoHundredDayAverage', 'N/A')
            },
            'dividend': {
                'dividend_rate': info.get('dividendRate', 'N/A'),
                'dividend_yield': info.get('dividendYield', 'N/A'),
                'ex_dividend_date': info.get('exDividendDate', 'N/A')
            }
        }
        
        return jsonify(stock_data), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve stock data: {str(e)}'
        }), 500


if __name__ == '__main__':
    app.run(debug=False, port=5000)

# Testing with curl:
# curl http://localhost:5000/api/stock/AAPL
# curl http://localhost:5000/api/stock/TSLA
# curl http://localhost:5000/api/stock/NVDA


## 3. Historical Market Data Endpoint

This endpoint accepts a JSON payload via POST request and returns historical market data (OHLCV) for a specified company symbol within a given date range.

### Endpoint Details:
| Property | Value |
|----------|-------|
| **Route** | `/api/historical` |
| **Method** | POST |
| **Content-Type** | application/json |
| **Input** | JSON with symbol, start_date, end_date |
| **Output** | JSON with historical OHLCV data |

### Request Body:
- **symbol**: Stock ticker symbol (required)
- **start_date**: Start date in YYYY-MM-DD format (required)
- **end_date**: End date in YYYY-MM-DD format (required)
- **interval**: Data interval - 1d, 5d, 1wk, 1mo, 3mo (optional, default: 1d)


In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
from datetime import datetime
import pandas as pd

app = Flask(__name__)

@app.route('/api/historical', methods=['POST'])
def get_historical_data():
    """
    Retrieve historical market data for a given symbol and date range.
    
    Expected JSON payload:
    {
        "symbol": "AAPL",
        "start_date": "2024-01-01",
        "end_date": "2024-12-31",
        "interval": "1d" (optional)
    }
    
    Returns:
        JSON response with historical OHLCV data
    """
    try:
        # Get JSON data from request
        data = request.get_json()
        
        # Validate required fields
        if not data:
            return jsonify({
                'success': False,
                'error': 'No JSON data provided'
            }), 400
        
        symbol = data.get('symbol')
        start_date = data.get('start_date')
        end_date = data.get('end_date')
        interval = data.get('interval', '1d')  # Default to daily interval
        
        # Validate required fields
        if not symbol:
            return jsonify({
                'success': False,
                'error': 'Symbol is required'
            }), 400
            
        if not start_date or not end_date:
            return jsonify({
                'success': False,
                'error': 'Both start_date and end_date are required'
            }), 400
        
        # Validate date formats
        try:
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            return jsonify({
                'success': False,
                'error': 'Invalid date format. Use YYYY-MM-DD'
            }), 400
        
        # Validate date range
        if start_dt > end_dt:
            return jsonify({
                'success': False,
                'error': 'start_date must be before end_date'
            }), 400
        
        # Validate interval
        valid_intervals = ['1d', '5d', '1wk', '1mo', '3mo']
        if interval not in valid_intervals:
            return jsonify({
                'success': False,
                'error': f'Invalid interval. Valid options: {valid_intervals}'
            }), 400
        
        # Create Ticker object and fetch historical data
        ticker = yf.Ticker(symbol.upper())
        history = ticker.history(start=start_date, end=end_date, interval=interval)
        
        # Check if data was returned
        if history.empty:
            return jsonify({
                'success': False,
                'error': f'No historical data found for {symbol} in the specified date range'
            }), 404
        
        # Convert DataFrame to list of dictionaries
        historical_data = []
        for date, row in history.iterrows():
            historical_data.append({
                'date': date.strftime('%Y-%m-%d'),
                'open': round(row['Open'], 2),
                'high': round(row['High'], 2),
                'low': round(row['Low'], 2),
                'close': round(row['Close'], 2),
                'volume': int(row['Volume']),
                'dividends': round(row.get('Dividends', 0), 4),
                'stock_splits': row.get('Stock Splits', 0)
            })
        
        # Build response
        response = {
            'success': True,
            'symbol': symbol.upper(),
            'start_date': start_date,
            'end_date': end_date,
            'interval': interval,
            'total_records': len(historical_data),
            'data': historical_data
        }
        
        return jsonify(response), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve historical data: {str(e)}'
        }), 500


if __name__ == '__main__':
    app.run(debug=False, port=5000)

# Testing with curl:
# curl -X POST http://localhost:5000/api/historical \
#   -H "Content-Type: application/json" \
#   -d '{"symbol": "AAPL", "start_date": "2024-01-01", "end_date": "2024-01-31"}'

# With interval:
# curl -X POST http://localhost:5000/api/historical \
#   -H "Content-Type: application/json" \
#   -d '{"symbol": "GOOGL", "start_date": "2024-01-01", "end_date": "2024-12-31", "interval": "1wk"}'


## 4. Analytical Insights Endpoint

This endpoint performs comprehensive analysis on the company's historical data and delivers actionable insights including:

- **Price Trend Analysis**: Overall trend direction and strength
- **Volatility Metrics**: Standard deviation, annualized volatility
- **Moving Averages**: SMA (Simple Moving Average) analysis
- **Performance Metrics**: Returns, drawdowns
- **Volume Analysis**: Volume trends and patterns
- **Support/Resistance Levels**: Key price levels

### Endpoint Details:
| Property | Value |
|----------|-------|
| **Route** | `/api/analysis` |
| **Method** | POST |
| **Content-Type** | application/json |
| **Input** | JSON with symbol, start_date, end_date |
| **Output** | JSON with comprehensive analysis |


In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np

app = Flask(__name__)


def calculate_moving_averages(df):
    """Calculate various moving averages."""
    result = {}
    if len(df) >= 10:
        result['sma_10'] = round(df['Close'].tail(10).mean(), 2)
    if len(df) >= 20:
        result['sma_20'] = round(df['Close'].tail(20).mean(), 2)
    if len(df) >= 50:
        result['sma_50'] = round(df['Close'].tail(50).mean(), 2)
    if len(df) >= 200:
        result['sma_200'] = round(df['Close'].tail(200).mean(), 2)
    return result


def calculate_volatility(df):
    """Calculate volatility metrics."""
    daily_returns = df['Close'].pct_change().dropna()
    return {
        'daily_volatility': round(daily_returns.std() * 100, 2),
        'annualized_volatility': round(daily_returns.std() * np.sqrt(252) * 100, 2),
        'max_daily_gain': round(daily_returns.max() * 100, 2),
        'max_daily_loss': round(daily_returns.min() * 100, 2)
    }


def calculate_performance(df):
    """Calculate performance metrics."""
    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]
    highest_price = df['High'].max()
    lowest_price = df['Low'].min()
    total_return = ((end_price - start_price) / start_price) * 100
    max_drawdown = ((lowest_price - highest_price) / highest_price) * 100
    return {
        'total_return_pct': round(total_return, 2),
        'start_price': round(start_price, 2),
        'end_price': round(end_price, 2),
        'highest_price': round(highest_price, 2),
        'lowest_price': round(lowest_price, 2),
        'max_drawdown_pct': round(max_drawdown, 2),
        'price_range': round(highest_price - lowest_price, 2)
    }


def analyze_trend(df):
    """Analyze price trend."""
    close_prices = df['Close'].values
    x = np.arange(len(close_prices))
    slope, _ = np.polyfit(x, close_prices, 1)
    
    if slope > 0.5:
        trend_direction = 'STRONGLY BULLISH'
    elif slope > 0.1:
        trend_direction = 'BULLISH'
    elif slope > -0.1:
        trend_direction = 'NEUTRAL'
    elif slope > -0.5:
        trend_direction = 'BEARISH'
    else:
        trend_direction = 'STRONGLY BEARISH'
    
    momentum = 0
    if len(df) >= 10:
        recent_avg = df['Close'].tail(5).mean()
        previous_avg = df['Close'].iloc[-10:-5].mean()
        momentum = ((recent_avg - previous_avg) / previous_avg) * 100
    
    return {
        'direction': trend_direction,
        'slope': round(slope, 4),
        'momentum_pct': round(momentum, 2)
    }


def analyze_volume(df):
    """Analyze volume patterns."""
    avg_volume = df['Volume'].mean()
    recent_volume = df['Volume'].tail(5).mean()
    volume_trend = ((recent_volume - avg_volume) / avg_volume) * 100
    return {
        'average_volume': int(avg_volume),
        'recent_average_volume': int(recent_volume),
        'volume_trend_pct': round(volume_trend, 2),
        'highest_volume_date': df['Volume'].idxmax().strftime('%Y-%m-%d'),
        'highest_volume': int(df['Volume'].max())
    }


def calculate_support_resistance(df):
    """Calculate support and resistance levels."""
    close_prices = df['Close']
    return {
        'strong_support': round(close_prices.quantile(0.1), 2),
        'support': round(close_prices.quantile(0.25), 2),
        'pivot': round(close_prices.median(), 2),
        'resistance': round(close_prices.quantile(0.75), 2),
        'strong_resistance': round(close_prices.quantile(0.9), 2)
    }


def generate_insights(performance, trend, volatility, volume, current_price, moving_averages):
    """Generate actionable insights based on analysis."""
    insights = []
    
    # Trend-based insights
    if 'BULLISH' in trend['direction']:
        insights.append(f"The stock shows a {trend['direction']} trend with a slope of {trend['slope']}.")
    elif 'BEARISH' in trend['direction']:
        insights.append(f"The stock shows a {trend['direction']} trend with a slope of {trend['slope']}.")
    else:
        insights.append("The stock is in a NEUTRAL/SIDEWAYS trend.")
    
    # Performance insights
    if performance['total_return_pct'] > 10:
        insights.append(f"Strong performance: {performance['total_return_pct']}% return in the period.")
    elif performance['total_return_pct'] < -10:
        insights.append(f"Weak performance: {performance['total_return_pct']}% return in the period.")
    
    # Volatility insights
    if volatility['annualized_volatility'] > 40:
        insights.append(f"HIGH VOLATILITY: {volatility['annualized_volatility']}% annualized. Consider risk management.")
    elif volatility['annualized_volatility'] < 20:
        insights.append(f"LOW VOLATILITY: {volatility['annualized_volatility']}% annualized. Relatively stable.")
    
    # Volume insights
    if volume['volume_trend_pct'] > 20:
        insights.append(f"Increasing volume (+{volume['volume_trend_pct']}%) suggests growing interest.")
    elif volume['volume_trend_pct'] < -20:
        insights.append(f"Decreasing volume ({volume['volume_trend_pct']}%) may indicate waning interest.")
    
    # Moving average insights
    if 'sma_50' in moving_averages and 'sma_200' in moving_averages:
        if moving_averages['sma_50'] > moving_averages['sma_200']:
            insights.append("GOLDEN CROSS: 50-day SMA is above 200-day SMA (bullish signal).")
        else:
            insights.append("DEATH CROSS: 50-day SMA is below 200-day SMA (bearish signal).")
    
    # Price vs SMA insights
    if 'sma_20' in moving_averages and current_price:
        if current_price > moving_averages['sma_20']:
            insights.append(f"Price (${current_price}) is above 20-day SMA (${moving_averages['sma_20']}) - short-term bullish.")
        else:
            insights.append(f"Price (${current_price}) is below 20-day SMA (${moving_averages['sma_20']}) - short-term bearish.")
    
    return insights


@app.route('/api/analysis', methods=['POST'])
def get_analysis():
    """Perform comprehensive analysis on historical stock data."""
    try:
        data = request.get_json()
        
        if not data:
            return jsonify({'success': False, 'error': 'No JSON data provided'}), 400
        
        symbol = data.get('symbol')
        start_date = data.get('start_date')
        end_date = data.get('end_date')
        
        if not all([symbol, start_date, end_date]):
            return jsonify({'success': False, 'error': 'symbol, start_date, and end_date are required'}), 400
        
        try:
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            return jsonify({'success': False, 'error': 'Invalid date format. Use YYYY-MM-DD'}), 400
        
        if start_dt > end_dt:
            return jsonify({'success': False, 'error': 'start_date must be before end_date'}), 400
        
        ticker = yf.Ticker(symbol.upper())
        history = ticker.history(start=start_date, end=end_date)
        
        if history.empty:
            return jsonify({'success': False, 'error': f'No historical data found for {symbol}'}), 404
        
        info = ticker.info
        current_price = info.get('regularMarketPrice', history['Close'].iloc[-1])
        
        performance = calculate_performance(history)
        trend = analyze_trend(history)
        volatility = calculate_volatility(history)
        volume = analyze_volume(history)
        support_resistance = calculate_support_resistance(history)
        moving_averages = calculate_moving_averages(history)
        insights = generate_insights(performance, trend, volatility, volume, current_price, moving_averages)
        
        response = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('shortName', 'N/A'),
            'analysis_period': {
                'start_date': start_date,
                'end_date': end_date,
                'trading_days': len(history)
            },
            'current_price': round(current_price, 2),
            'performance': performance,
            'trend_analysis': trend,
            'volatility_metrics': volatility,
            'volume_analysis': volume,
            'support_resistance_levels': support_resistance,
            'moving_averages': moving_averages,
            'insights': insights,
            'disclaimer': 'This analysis is for informational purposes only and should not be considered as financial advice.'
        }
        
        return jsonify(response), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to perform analysis: {str(e)}'
        }), 500


if __name__ == '__main__':
    app.run(debug=False, port=5000)

# Testing with curl:
# curl -X POST http://localhost:5000/api/analysis \
#   -H "Content-Type: application/json" \
#   -d '{"symbol": "AAPL", "start_date": "2024-01-01", "end_date": "2024-12-15"}'


## Complete API Application

Below is the complete Flask application combining all four endpoints into a single, runnable server:


In [ ]:
from flask import Flask, request, jsonify
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np

app = Flask(__name__)


# ============================================================================
# UTILITY FUNCTIONS FOR ANALYSIS
# ============================================================================

def calculate_moving_averages(df):
    """Calculate various moving averages."""
    result = {}
    if len(df) >= 10:
        result['sma_10'] = round(df['Close'].tail(10).mean(), 2)
    if len(df) >= 20:
        result['sma_20'] = round(df['Close'].tail(20).mean(), 2)
    if len(df) >= 50:
        result['sma_50'] = round(df['Close'].tail(50).mean(), 2)
    if len(df) >= 200:
        result['sma_200'] = round(df['Close'].tail(200).mean(), 2)
    return result


def calculate_volatility(df):
    """Calculate volatility metrics."""
    daily_returns = df['Close'].pct_change().dropna()
    return {
        'daily_volatility': round(daily_returns.std() * 100, 2),
        'annualized_volatility': round(daily_returns.std() * np.sqrt(252) * 100, 2),
        'max_daily_gain': round(daily_returns.max() * 100, 2),
        'max_daily_loss': round(daily_returns.min() * 100, 2)
    }


def calculate_performance(df):
    """Calculate performance metrics."""
    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]
    highest_price = df['High'].max()
    lowest_price = df['Low'].min()
    total_return = ((end_price - start_price) / start_price) * 100
    max_drawdown = ((lowest_price - highest_price) / highest_price) * 100
    return {
        'total_return_pct': round(total_return, 2),
        'start_price': round(start_price, 2),
        'end_price': round(end_price, 2),
        'highest_price': round(highest_price, 2),
        'lowest_price': round(lowest_price, 2),
        'max_drawdown_pct': round(max_drawdown, 2),
        'price_range': round(highest_price - lowest_price, 2)
    }


def analyze_trend(df):
    """Analyze price trend."""
    close_prices = df['Close'].values
    x = np.arange(len(close_prices))
    slope, _ = np.polyfit(x, close_prices, 1)
    
    if slope > 0.5:
        trend_direction = 'STRONGLY BULLISH'
    elif slope > 0.1:
        trend_direction = 'BULLISH'
    elif slope > -0.1:
        trend_direction = 'NEUTRAL'
    elif slope > -0.5:
        trend_direction = 'BEARISH'
    else:
        trend_direction = 'STRONGLY BEARISH'
    
    momentum = 0
    if len(df) >= 10:
        recent_avg = df['Close'].tail(5).mean()
        previous_avg = df['Close'].iloc[-10:-5].mean()
        momentum = ((recent_avg - previous_avg) / previous_avg) * 100
    
    return {
        'direction': trend_direction,
        'slope': round(slope, 4),
        'momentum_pct': round(momentum, 2)
    }


def analyze_volume(df):
    """Analyze volume patterns."""
    avg_volume = df['Volume'].mean()
    recent_volume = df['Volume'].tail(5).mean()
    volume_trend = ((recent_volume - avg_volume) / avg_volume) * 100
    return {
        'average_volume': int(avg_volume),
        'recent_average_volume': int(recent_volume),
        'volume_trend_pct': round(volume_trend, 2),
        'highest_volume_date': df['Volume'].idxmax().strftime('%Y-%m-%d'),
        'highest_volume': int(df['Volume'].max())
    }


def calculate_support_resistance(df):
    """Calculate support and resistance levels."""
    close_prices = df['Close']
    return {
        'strong_support': round(close_prices.quantile(0.1), 2),
        'support': round(close_prices.quantile(0.25), 2),
        'pivot': round(close_prices.median(), 2),
        'resistance': round(close_prices.quantile(0.75), 2),
        'strong_resistance': round(close_prices.quantile(0.9), 2)
    }


def generate_insights(performance, trend, volatility, volume, current_price, moving_averages):
    """Generate actionable insights based on analysis."""
    insights = []
    
    if 'BULLISH' in trend['direction']:
        insights.append(f"The stock shows a {trend['direction']} trend with a slope of {trend['slope']}.")
    elif 'BEARISH' in trend['direction']:
        insights.append(f"The stock shows a {trend['direction']} trend with a slope of {trend['slope']}.")
    else:
        insights.append("The stock is in a NEUTRAL/SIDEWAYS trend.")
    
    if performance['total_return_pct'] > 10:
        insights.append(f"Strong performance: {performance['total_return_pct']}% return in the period.")
    elif performance['total_return_pct'] < -10:
        insights.append(f"Weak performance: {performance['total_return_pct']}% return in the period.")
    
    if volatility['annualized_volatility'] > 40:
        insights.append(f"HIGH VOLATILITY: {volatility['annualized_volatility']}% annualized. Consider risk management.")
    elif volatility['annualized_volatility'] < 20:
        insights.append(f"LOW VOLATILITY: {volatility['annualized_volatility']}% annualized. Relatively stable.")
    
    if volume['volume_trend_pct'] > 20:
        insights.append(f"Increasing volume (+{volume['volume_trend_pct']}%) suggests growing interest.")
    elif volume['volume_trend_pct'] < -20:
        insights.append(f"Decreasing volume ({volume['volume_trend_pct']}%) may indicate waning interest.")
    
    if 'sma_50' in moving_averages and 'sma_200' in moving_averages:
        if moving_averages['sma_50'] > moving_averages['sma_200']:
            insights.append("GOLDEN CROSS: 50-day SMA is above 200-day SMA (bullish signal).")
        else:
            insights.append("DEATH CROSS: 50-day SMA is below 200-day SMA (bearish signal).")
    
    if 'sma_20' in moving_averages and current_price:
        if current_price > moving_averages['sma_20']:
            insights.append(f"Price (${current_price}) is above 20-day SMA (${moving_averages['sma_20']}) - short-term bullish.")
        else:
            insights.append(f"Price (${current_price}) is below 20-day SMA (${moving_averages['sma_20']}) - short-term bearish.")
    
    return insights


# ============================================================================
# API ENDPOINTS
# ============================================================================

@app.route('/')
def home():
    """Home endpoint with API documentation."""
    return jsonify({
        'message': 'Welcome to Stock Market API',
        'version': '1.0',
        'endpoints': {
            'company_info': {
                'url': '/api/company/<symbol>',
                'method': 'GET',
                'description': 'Get detailed company information'
            },
            'stock_data': {
                'url': '/api/stock/<symbol>',
                'method': 'GET',
                'description': 'Get real-time stock market data'
            },
            'historical_data': {
                'url': '/api/historical',
                'method': 'POST',
                'description': 'Get historical market data for a date range'
            },
            'analysis': {
                'url': '/api/analysis',
                'method': 'POST',
                'description': 'Get comprehensive analysis and insights'
            }
        }
    })


@app.route('/api/company/<symbol>', methods=['GET'])
def get_company_info(symbol):
    """Retrieve detailed company information for a given stock symbol."""
    try:
        ticker = yf.Ticker(symbol.upper())
        info = ticker.info
        
        if not info or 'shortName' not in info:
            return jsonify({
                'success': False,
                'error': f'No data found for symbol: {symbol}'
            }), 404
        
        officers = []
        if 'companyOfficers' in info:
            for officer in info['companyOfficers'][:5]:
                officers.append({
                    'name': officer.get('name', 'N/A'),
                    'title': officer.get('title', 'N/A'),
                    'age': officer.get('age', 'N/A')
                })
        
        company_data = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('longName', info.get('shortName', 'N/A')),
            'business_summary': info.get('longBusinessSummary', 'N/A'),
            'industry': info.get('industry', 'N/A'),
            'sector': info.get('sector', 'N/A'),
            'website': info.get('website', 'N/A'),
            'headquarters': {
                'city': info.get('city', 'N/A'),
                'state': info.get('state', 'N/A'),
                'country': info.get('country', 'N/A')
            },
            'employees': info.get('fullTimeEmployees', 'N/A'),
            'key_officers': officers
        }
        
        return jsonify(company_data), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve company information: {str(e)}'
        }), 500


@app.route('/api/stock/<symbol>', methods=['GET'])
def get_stock_data(symbol):
    """Fetch real-time stock market data for a given symbol."""
    try:
        ticker = yf.Ticker(symbol.upper())
        info = ticker.info
        
        if not info or 'regularMarketPrice' not in info:
            return jsonify({
                'success': False,
                'error': f'No stock data found for symbol: {symbol}'
            }), 404
        
        current_price = info.get('regularMarketPrice', 0)
        previous_close = info.get('regularMarketPreviousClose', 0)
        price_change = round(current_price - previous_close, 2) if current_price and previous_close else 0
        percentage_change = round((price_change / previous_close) * 100, 2) if previous_close else 0
        
        stock_data = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('shortName', 'N/A'),
            'market_state': info.get('marketState', 'N/A'),
            'currency': info.get('currency', 'USD'),
            'price_data': {
                'current_price': current_price,
                'previous_close': previous_close,
                'price_change': price_change,
                'percentage_change': percentage_change,
                'day_high': info.get('dayHigh', 'N/A'),
                'day_low': info.get('dayLow', 'N/A'),
                'open_price': info.get('regularMarketOpen', 'N/A')
            },
            'volume_data': {
                'volume': info.get('regularMarketVolume', 'N/A'),
                'average_volume': info.get('averageVolume', 'N/A'),
                'average_volume_10day': info.get('averageVolume10days', 'N/A')
            },
            'valuation': {
                'market_cap': info.get('marketCap', 'N/A'),
                'enterprise_value': info.get('enterpriseValue', 'N/A'),
                'pe_ratio': info.get('trailingPE', 'N/A'),
                'forward_pe': info.get('forwardPE', 'N/A'),
                'price_to_book': info.get('priceToBook', 'N/A')
            },
            'range': {
                'fifty_two_week_high': info.get('fiftyTwoWeekHigh', 'N/A'),
                'fifty_two_week_low': info.get('fiftyTwoWeekLow', 'N/A'),
                'fifty_day_average': info.get('fiftyDayAverage', 'N/A'),
                'two_hundred_day_average': info.get('twoHundredDayAverage', 'N/A')
            },
            'dividend': {
                'dividend_rate': info.get('dividendRate', 'N/A'),
                'dividend_yield': info.get('dividendYield', 'N/A'),
                'ex_dividend_date': info.get('exDividendDate', 'N/A')
            }
        }
        
        return jsonify(stock_data), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve stock data: {str(e)}'
        }), 500


@app.route('/api/historical', methods=['POST'])
def get_historical_data():
    """Retrieve historical market data for a given symbol and date range."""
    try:
        data = request.get_json()
        
        if not data:
            return jsonify({
                'success': False,
                'error': 'No JSON data provided'
            }), 400
        
        symbol = data.get('symbol')
        start_date = data.get('start_date')
        end_date = data.get('end_date')
        interval = data.get('interval', '1d')
        
        if not symbol:
            return jsonify({'success': False, 'error': 'Symbol is required'}), 400
        if not start_date or not end_date:
            return jsonify({'success': False, 'error': 'Both start_date and end_date are required'}), 400
        
        try:
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            return jsonify({'success': False, 'error': 'Invalid date format. Use YYYY-MM-DD'}), 400
        
        if start_dt > end_dt:
            return jsonify({'success': False, 'error': 'start_date must be before end_date'}), 400
        
        valid_intervals = ['1d', '5d', '1wk', '1mo', '3mo']
        if interval not in valid_intervals:
            return jsonify({'success': False, 'error': f'Invalid interval. Valid options: {valid_intervals}'}), 400
        
        ticker = yf.Ticker(symbol.upper())
        history = ticker.history(start=start_date, end=end_date, interval=interval)
        
        if history.empty:
            return jsonify({
                'success': False,
                'error': f'No historical data found for {symbol} in the specified date range'
            }), 404
        
        historical_data = []
        for date, row in history.iterrows():
            historical_data.append({
                'date': date.strftime('%Y-%m-%d'),
                'open': round(row['Open'], 2),
                'high': round(row['High'], 2),
                'low': round(row['Low'], 2),
                'close': round(row['Close'], 2),
                'volume': int(row['Volume']),
                'dividends': round(row.get('Dividends', 0), 4),
                'stock_splits': row.get('Stock Splits', 0)
            })
        
        response = {
            'success': True,
            'symbol': symbol.upper(),
            'start_date': start_date,
            'end_date': end_date,
            'interval': interval,
            'total_records': len(historical_data),
            'data': historical_data
        }
        
        return jsonify(response), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to retrieve historical data: {str(e)}'
        }), 500


@app.route('/api/analysis', methods=['POST'])
def get_analysis():
    """Perform comprehensive analysis on historical stock data."""
    try:
        data = request.get_json()
        
        if not data:
            return jsonify({'success': False, 'error': 'No JSON data provided'}), 400
        
        symbol = data.get('symbol')
        start_date = data.get('start_date')
        end_date = data.get('end_date')
        
        if not all([symbol, start_date, end_date]):
            return jsonify({'success': False, 'error': 'symbol, start_date, and end_date are required'}), 400
        
        try:
            start_dt = datetime.strptime(start_date, '%Y-%m-%d')
            end_dt = datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            return jsonify({'success': False, 'error': 'Invalid date format. Use YYYY-MM-DD'}), 400
        
        if start_dt > end_dt:
            return jsonify({'success': False, 'error': 'start_date must be before end_date'}), 400
        
        ticker = yf.Ticker(symbol.upper())
        history = ticker.history(start=start_date, end=end_date)
        
        if history.empty:
            return jsonify({'success': False, 'error': f'No historical data found for {symbol}'}), 404
        
        info = ticker.info
        current_price = info.get('regularMarketPrice', history['Close'].iloc[-1])
        
        performance = calculate_performance(history)
        trend = analyze_trend(history)
        volatility = calculate_volatility(history)
        volume = analyze_volume(history)
        support_resistance = calculate_support_resistance(history)
        moving_averages = calculate_moving_averages(history)
        insights = generate_insights(performance, trend, volatility, volume, current_price, moving_averages)
        
        response = {
            'success': True,
            'symbol': symbol.upper(),
            'company_name': info.get('shortName', 'N/A'),
            'analysis_period': {
                'start_date': start_date,
                'end_date': end_date,
                'trading_days': len(history)
            },
            'current_price': round(current_price, 2),
            'performance': performance,
            'trend_analysis': trend,
            'volatility_metrics': volatility,
            'volume_analysis': volume,
            'support_resistance_levels': support_resistance,
            'moving_averages': moving_averages,
            'insights': insights,
            'disclaimer': 'This analysis is for informational purposes only and should not be considered as financial advice.'
        }
        
        return jsonify(response), 200
        
    except Exception as e:
        return jsonify({
            'success': False,
            'error': f'Failed to perform analysis: {str(e)}'
        }), 500


if __name__ == '__main__':
    app.run(debug=False, port=5000)


# ============================================================================
# TESTING THE API
# ============================================================================
#
# 1. Company Information:
#    curl http://localhost:5000/api/company/AAPL
#
# 2. Stock Market Data:
#    curl http://localhost:5000/api/stock/TSLA
#
# 3. Historical Data:
#    curl -X POST http://localhost:5000/api/historical \
#      -H "Content-Type: application/json" \
#      -d '{"symbol": "AAPL", "start_date": "2024-01-01", "end_date": "2024-01-31"}'
#
# 4. Analysis:
#    curl -X POST http://localhost:5000/api/analysis \
#      -H "Content-Type: application/json" \
#      -d '{"symbol": "NVDA", "start_date": "2024-01-01", "end_date": "2024-12-15"}'


## Testing the API with Python Requests

After starting the Flask server (run the cell above), you can test the endpoints using the `requests` library:


In [ ]:
# import requests
# import json

# BASE_URL = 'http://localhost:5000'

# # Test 1: Get Company Information
# print("=" * 60)
# print("TEST 1: Company Information Endpoint")
# print("=" * 60)
# response = requests.get(f"{BASE_URL}/api/company/AAPL")
# print(json.dumps(response.json(), indent=2))


In [ ]:
# # Test 2: Get Stock Market Data
# print("\n" + "=" * 60)
# print("TEST 2: Stock Market Data Endpoint")
# print("=" * 60)
# response = requests.get(f"{BASE_URL}/api/stock/NVDA")
# print(json.dumps(response.json(), indent=2))


In [ ]:
# # Test 3: Get Historical Data
# print("\n" + "=" * 60)
# print("TEST 3: Historical Market Data Endpoint")
# print("=" * 60)
# payload = {
#     "symbol": "GOOGL",
#     "start_date": "2024-12-01",
#     "end_date": "2024-12-15",
#     "interval": "1d"
# }
# response = requests.post(f"{BASE_URL}/api/historical", json=payload)
# print(json.dumps(response.json(), indent=2))


In [ ]:
# # Test 4: Get Analysis
# print("\n" + "=" * 60)
# print("TEST 4: Analytical Insights Endpoint")
# print("=" * 60)
# payload = {
#     "symbol": "MSFT",
#     "start_date": "2024-01-01",
#     "end_date": "2024-12-15"
# }
# response = requests.post(f"{BASE_URL}/api/analysis", json=payload)
# print(json.dumps(response.json(), indent=2))


## Summary

In this notebook, we developed four robust API endpoints:

| Endpoint | Method | Route | Description |
|----------|--------|-------|-------------|
| **Company Info** | GET | `/api/company/<symbol>` | Company details, officers, business summary |
| **Stock Data** | GET | `/api/stock/<symbol>` | Real-time price, volume, valuation metrics |
| **Historical Data** | POST | `/api/historical` | OHLCV data for a date range |
| **Analysis** | POST | `/api/analysis` | Comprehensive analysis with actionable insights |

### Key Features:
- **Error Handling**: Proper validation and error responses
- **Data Validation**: Date format, symbol validity, required fields
- **Comprehensive Analysis**: Trend, volatility, moving averages, support/resistance
- **Actionable Insights**: Automated generation of trading signals and observations

### Technologies Used:
- **Flask**: Web framework for building RESTful APIs
- **yfinance**: Yahoo Finance API wrapper for market data
- **pandas/numpy**: Data manipulation and numerical analysis

---

**Note**: The yfinance library fetches data from Yahoo Finance. Some data may be delayed or unavailable depending on the market and your location. Always verify critical data with official sources.


<!--  -->